In [53]:
!pip install imitation

  Using cached imitation-1.0.0-py3-none-any.whl.metadata (14 kB)
  Using cached seals-0.2.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached sacred-0.8.5-py2.py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_sb3-3.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached optuna-3.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached xxhash-3.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl.metadata (28 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Using cached munch-4.0.0-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached imitation-1.0.0-py3-none-any.whl (216 kB)
   --------------------------------------

In [36]:
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

In [6]:
data_folder = os.path.join('data_copy')

In [49]:
# Define a mapping of body parts to column ranges
body_parts_mapping = {
    'Head': (0, 1),
    'SpineShoulders': (3, 4),
    'SpineMid': (6, 7),
    'LeftShoulder': (9, 10),
    'RightShoulder': (12, 13),
    'LeftElbow': (15, 16),
    'RightElbow': (18, 19),
    'LeftWrist': (21, 22),
    'RightWrist': (24, 25),
    'SpineBase': (27, 28),
    'LeftHip': (30, 31),
    'RightHip': (33, 34),
    'LeftKnee': (36, 37),
    'RightKnee': (39, 40),
    'LeftAnkle': (42, 43),
    'RightAnkle': (45, 46),
}

In [50]:
# Function to preprocess data
def preprocess_data(data_folder):
    joint_positions = []
    actions = []

    exercise_folders = os.listdir(data_folder)
    exercise_folders.remove('guide')
    exercise_folders.remove('plot') 

    # Define the output folder path
    output_folder = os.path.join("States and Actions")
    os.makedirs(output_folder, exist_ok=True)

    for exercise_folder in exercise_folders:

        exercise_path = os.path.join(data_folder, exercise_folder)
        exercise_path_list = os.listdir(exercise_path)
        
        for typedata_folder in exercise_path_list:
        
            typedata_path = os.path.join(exercise_path, typedata_folder)
            typedata_path_list = os.listdir(typedata_path)

            for typepatient_folder in typedata_path_list:

                typepatient_path = os.path.join(typedata_path, typepatient_folder)
                typepatient_path_list = os.listdir(typepatient_path)

                for patient_folder in typepatient_path_list:

                    patient_path = os.path.join(typepatient_path, patient_folder)
                    patient_path_list = os.listdir(patient_path)

                    for trial_file in patient_path_list:
                        if not trial_file.startswith('.'):  # Skip files/directories starting with a period
                            file_path = os.path.join(patient_path, trial_file)
                            joints_data = read_txt_file(file_path)
        
                            # Extract relevant columns based on body parts mapping
                            extracted_data = extract_body_parts(joints_data)
                            
                            # Print metadata
                            print('-----------------------------------------------')
                            print('TypePatient -', typepatient_folder)
                            print('Exercise -', exercise_folder)
                            print('Typedata -', typedata_folder)
                            print('Patient -', patient_folder)
                            print('Trial -', trial_file)
                            
                            # Process each instance and generate actions
                            instance_actions = generate_actions(extracted_data)
                            actions.extend(instance_actions)

                            # Save the joint positions
                            joint_positions_output_folder = os.path.join(output_folder, typepatient_folder, exercise_folder, typedata_folder, patient_folder)
                            os.makedirs(joint_positions_output_folder, exist_ok=True)
                            joint_positions_file = os.path.join(joint_positions_output_folder, f"{trial_file}_joint_positions.txt")
                            with open(joint_positions_file, 'w') as file:
                                for instance in extracted_data:
                                    file.write(f"Instance:\n")
                                    for joint, coords in instance.items():
                                        file.write(f"{joint}: {coords}\n")
                                    file.write("\n")
                            
                            # Save the actions as text file
                            actions_output_folder = os.path.join(output_folder, typepatient_folder, exercise_folder, typedata_folder, patient_folder)
                            os.makedirs(actions_output_folder, exist_ok=True)
                            actions_file = os.path.join(actions_output_folder, f"{trial_file}_actions.txt")
                            with open(actions_file, 'w') as file:
                                for action in instance_actions:
                                    file.write(f"{action[0]} {action[1]}\n")

                            joint_positions.append(extracted_data)
    
     # Split data into training and testing sets
    train_data, test_data = train_test_split(joint_positions, test_size=0.2, random_state=42)
    train_actions, test_actions = train_test_split( actions, test_size=0.2, random_state=42)
    
    print('The End')
    
    return train_data, test_data, train_actions, test_actions


# Function to read data from a text file
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        joints_data = [list(map(float, line.split(';'))) for line in lines]
    return joints_data

# Function to extract relevant body parts from joint data
def extract_body_parts(joints_data):
    num_instances = len(joints_data)
    extracted_data = []

    # Iterate over instances
    for instance_idx in range(num_instances):
        instance_data = {}
        # Iterate over body parts
        for body_part, (start_col, end_col) in body_parts_mapping.items():
            body_part_data = joints_data[instance_idx][start_col:end_col + 1]
            instance_data[body_part] = body_part_data
        extracted_data.append(instance_data)

    return extracted_data

# Function to generate grid position from joint coordinates
def generate_grid(joint_coords):
    # Define grid size (adjust as needed)
    grid_size = (10, 10)
    # Determine grid position of the joint coordinates
    grid_x = int(joint_coords[0] * grid_size[0])
    grid_y = int(joint_coords[1] * grid_size[1])
    return grid_x, grid_y

# Function to generate actions based on joint positions
def generate_actions(extracted_data):
    actions = []
    instance_idx = 1  # Track instance number
    for i in range(0, len(extracted_data), 1):  # Generate actions for every other instance
        if i + 1 < len(extracted_data):
            # Extract wrist grid positions for current and next instances
            current_left_wrist = extracted_data[i]['LeftWrist']
            next_left_wrist = extracted_data[i + 1]['LeftWrist']
            current_right_wrist = extracted_data[i]['RightWrist']
            next_right_wrist = extracted_data[i + 1]['RightWrist']
            
            # Compare grid positions to determine movement direction for left wrist
            left_movement = compare_positions(current_left_wrist, next_left_wrist)
            # Compare grid positions to determine movement direction for right wrist
            right_movement = compare_positions(current_right_wrist, next_right_wrist)
            
            # Map grid position comparison to action
            left_action = f"{left_movement} (from {current_left_wrist} to {next_left_wrist})" if left_movement != "Hold" else "Hold"
            right_action = f"{right_movement} (from {current_right_wrist} to {next_right_wrist})" if right_movement != "Hold" else "Hold"
            
            # Print the action for the current instance range
            print(f"Action from instance {instance_idx} to {instance_idx+2}: Left- {left_action}, Right- {right_action}")
            
            actions.append((left_action, right_action))
            instance_idx += 1  # Move to the next instance
    return actions


# Function to compare grid positions and determine movement direction
def compare_positions(current_grid, next_grid):
    # Compare grid positions between current and next coordinates
    # Return movement direction based on the comparison result
    # Example implementation (adjust as needed)
    if current_grid[0] == next_grid[0] and current_grid[1] == next_grid[1]:
        return "Hold"
    elif current_grid[0] < next_grid[0]:
        if current_grid[1] < next_grid[1]:
            return "DownRight"
        elif current_grid[1] > next_grid[1]:
            return "UpRight"
        else:
            return "Right"
    elif current_grid[0] > next_grid[0]:
        if current_grid[1] < next_grid[1]:
            return "DownLeft"
        elif current_grid[1] > next_grid[1]:
            return "UpLeft"
        else:
            return "Left"
    else:
        if current_grid[1] < next_grid[1]:
            return "Down"
        elif current_grid[1] > next_grid[1]:
            return "Up"
    return "no movement detected"  # Default action if no movement detected

In [64]:
# Step 2: Combine joint coordinates into a single observation vector
class PatientHealthEnv(Env):
    def __init__(self, train_data, train_actions, test_data, test_actions):
        self.train_data = train_data
        self.train_actions = train_actions
        self.test_data = test_data
        self.test_actions = test_actions
        self.current_step = 0
        self.action_space = Discrete(len(train_actions))
        self.observation_space = Box(low=0, high=100, shape=(len(train_data[0]),))

    #def reset(self):
    #    #self.current_step = 0
        ##obs_dict = self.train_data[self.current_step]
        ##obs_values = np.concatenate([np.array(list(coords.values())) for coords in obs_dict])
        #obs = obs_values.flatten()
        #print("Observation shape: ", obs.shape)
        #return obs

    def reset(self):
        self.current_step = 0
        obs = np.array(self.train_data[self.current_step]).flatten()
        print("Observation shape:", obs.shape)
        print("Observation:", obs)
        return obs


    def step(self, action):
        obs = self.train_data[self.current_step]
        reward = self.evaluate_action(action)
        self.current_step += 1
        done = self.current_step >= len(self.train_data)
        info = {}  # Additional information (if needed)
        return obs, reward, done, info

    def evaluate_action(self, action):
        predicted_action = self.train_actions[self.current_step]
        ground_truth_action = self.test_actions[self.current_step]
        if predicted_action == ground_truth_action:
            return 1  # Correct action
        else:
            return -1  # Incorrect action

    def seed(self, seed=None):
        np.random.seed(seed)

In [39]:
# Preprocess data
train_data, test_data, train_actions, test_actions = preprocess_data(data_folder)

-----------------------------------------------
TypePatient - affected
Exercise - E1
Typedata - capture
Patient - P01
Trial - E1_filt_captured_P01_affected_1.txt
Action from instance 1 to 3: Left-  Hold, Right-  Hold
Action from instance 3 to 5: Left-  Hold, Right-  Hold
Action from instance 5 to 7: Left-  Down, Right-  Hold
Action from instance 7 to 9: Left-  Right, Right-  Hold
Action from instance 9 to 11: Left-  Hold, Right-  Hold
Action from instance 11 to 13: Left-  Hold, Right-  Hold
Action from instance 13 to 15: Left-  Up, Right-  Hold
Action from instance 15 to 17: Left-  Left, Right-  Hold
Action from instance 17 to 19: Left-  Up, Right-  Hold
Action from instance 19 to 21: Left-  Hold, Right-  Hold
Action from instance 21 to 23: Left-  Up, Right-  Hold
Action from instance 23 to 25: Left-  Hold, Right-  Hold
Action from instance 25 to 27: Left-  Hold, Right-  Hold
Action from instance 27 to 29: Left-  Hold, Right-  Hold
-----------------------------------------------
TypePa

KeyboardInterrupt: 

In [55]:
# Print the sizes of the train and test datasets
print("Size of train dataset:", len(train_data))
print("Size of test dataset:", len(test_data))
print("Size of train actions dataset:", len(train_actions))
print("Size of test actions dataset:", len(test_actions))

Size of train dataset: 1214
Size of test dataset: 304
Size of train actions dataset: 89039
Size of test actions dataset: 22260


In [65]:
# Parallel environments
vec_env = make_vec_env(PatientHealthEnv, n_envs=4, env_kwargs={'train_data': train_data,
                                                               'train_actions': train_actions,
                                                               'test_data': test_data,
                                                               'test_actions': test_actions})
# Train a PPO model on the environment
model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_patient_health_env")

# Load the trained model
model = PPO.load("ppo_patient_health_env")

# Perform inference using the trained model
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

Using cpu device
Observation shape: (98,)
Observation: [{'Head': [0.33745, -0.18372], 'SpineShoulders': [0.38181, -0.3511], 'SpineMid': [0.355595, -0.578705], 'LeftShoulder': [0.38664499999999996, -0.446795], 'RightShoulder': [0.43796999999999997, -0.36074], 'LeftElbow': [0.23919, -0.551275], 'RightElbow': [0.41788000000000003, -0.55088], 'LeftWrist': [0.155175, -0.7674300000000001], 'RightWrist': [0.17593499999999998, -0.5777899999999999], 'SpineBase': [0.31637, -0.8873], 'LeftHip': [0.28951, -0.8906350000000001], 'RightHip': [0.333595, -0.8601850000000001], 'LeftKnee': [-0.0165375, -0.78771], 'RightKnee': [0.031070999999999998, -0.8035399999999999], 'LeftAnkle': [0.08780249999999999, -1.2386], 'RightAnkle': [0.060361, -1.14065]}
 {'Head': [0.337406666666667, -0.18374000000000001], 'SpineShoulders': [0.3817866666666671, -0.35111], 'SpineMid': [0.355475, -0.578785], 'LeftShoulder': [0.3866416666666671, -0.44678500000000004], 'RightShoulder': [0.43799999999999994, -0.3608233333333329], 

TypeError: float() argument must be a string or a real number, not 'dict'

In [57]:
obs.shape

NameError: name 'obs' is not defined